# $\text{Read Data}$

In [6]:
import pandas as pd

# === set your path ===
df = pd.read_csv("prod_market_m.csv", low_memory=False, encoding="utf-8")
pd.set_option('display.max_columns', None)
df

,store,month_idx,prod_key,prod_type,prod_id,year52,m4,month_label,total_packs,total_rev,menthol,dlx,special,supslim,slim,single,carton,pack_kw,value,generic_automated,generic_hardcoded,flavored,premium,cigarettes,ok,sale,tar_mean,nic_mean,co_mean,income,educ,hsizeavg,age9,age60,ethnic,nocar,custcount,implied discount,brand,brand_clean,size,size_carton_pack,upc_norm,packs_per_item_wavg,avg_pack_price,market_size_month,prod_mkt_share
0,2,1,cigarette|1100015308,cigarette,1100015308,1,1,Y1-M1,30.0,44.55,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,14.527473,0.973626,13.086957,10.553206,0.248935,2.531062,0.117509,0.232865,0.11428,0.124603,2127.0,0.0,Generic,generic,King,10 PK,1100015308,10.0,1.485,26374.5,0.001137
1,2,1,cigarette|1230010113,cigarette,1230010113,1,1,Y1-M1,20.0,29.70,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,14.527473,0.973626,13.086957,10.553206,0.248935,2.531062,0.117509,0.232865,0.11428,0.124603,1940.0,0.0,Generic,generic,King,10 PK,1230010113,10.0,1.485,26374.5,0.000758
2,2,1,cigarette|1230010213,cigarette,1230010213,1,1,Y1-M1,10.0,14.85,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,14.527473,0.973626,13.086957,10.553206,0.248935,2.531062,0.117509,0.232865,0.11428,0.124603,1940.0,0.0,Generic,generic,King,10 PK,1230010213,10.0,1.485,26374.5,0.000379
3,2,1,cigarette|1230010413,cigarette,1230010413,1,1,Y1-M1,90.0,133.65,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,14.527473,0.973626,13.086957,10.553206,0.248935,2.531062,0.117509,0.232865,0.11428,0.124603,1981.0,0.0,Generic,generic,King,10 PK,1230010413,10.0,1.485,26374.5,0.003412
4,2,1,cigarette|1230011013,cigarette,1230011013,1,1,Y1-M1,80.0,118.80,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,14.527473,0.973626,13.086957,10.553206,0.248935,2.531062,0.117509,0.232865,0.11428,0.124603,1981.0,0.0,Generic,generic,King,10 PK,1230011013,10.0,1.485,26374.5,0.003033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731765,146,100,cigarette|2610000354,cigarette,2610000354,8,9,Y8-M9,1.0,2.82,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,15.000000,0.900000,13.121951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1764.0,0.0,Generic,generic,Reg,1 CT,2610000354,1.0,2.820,2460.0,0.000407
731766,146,100,cigarette|2610000364,cigarette,2610000364,8,9,Y8-M9,4.0,11.28,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,15.000000,0.900000,13.121951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2109.0,0.0,Generic,generic,Reg,1 CT,2610000364,1.0,2.820,2460.0,0.001626
731767,146,100,cigarette|2610000504,cigarette,2610000504,8,9,Y8-M9,1.0,2.82,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,15.000000,0.900000,13.121951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2109.0,0.0,Generic,generic,Reg,1 CT,2610000504,1.0,2.820,2460.0,0.000407
731768,146,100,cigarette|2610000644,cigarette,2610000644,8,9,Y8-M9,5.0,14.10,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,15.000000,0.900000,13.121951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2109.0,0.0,Generic,generic,Reg,1 CT,2610000644,1.0,2.820,2460.0,0.002033


# $\text{Statistics}$

In [8]:
def series_flag(df, col, dtype="int8"):
    if col in df.columns:
        return pd.to_numeric(df[col], errors="coerce").fillna(0).astype(dtype)
    return pd.Series(0, index=df.index, dtype=dtype)

# ========================
# 0) data & keys
# ========================
df = df.rename(columns={"avg_pack_price": "price"})
mkt = ["store","month_idx"]

sum_inside = df.groupby(mkt, observed=True)["prod_mkt_share"].transform("sum")
df = df[(df["prod_mkt_share"] > 0) & (sum_inside < 1)].copy()
df["s0"]    = np.clip(1.0 - sum_inside, 1e-12, 1 - 1e-12)
df["log_s"] = np.log(df["prod_mkt_share"]) - np.log(df["s0"])

if "prod_id" not in df.columns:
    if "upc_norm" in df.columns:
        df["prod_id"] = df["upc_norm"].astype("string")
    elif "upc" in df.columns:
        df["prod_id"] = df["upc"].astype("string").str.replace(r"\D","", regex=True)
    else:
        df["prod_id"] = df.index.astype("string")

brand_raw = df.get("brand", pd.Series("", index=df.index))
brand_key = (brand_raw.astype("string").str.strip().str.lower()
             .mask(lambda s: s.eq("") | s.isna(), "generic"))
is_generic = (series_flag(df, "generic_hardcoded") > 0) | brand_key.eq("generic")
brand_key  = brand_key.mask(is_generic, "generic")

# ========================
# 1) regressors
# ========================
dummy_cols = [c for c in ["dlx","supslim","slim","value","premium","flavored","carton"] if c in df.columns]
cont_cols  = [c for c in ["tar_mean","nic_mean","co_mean"] if c in df.columns]
Xnames     = ["price"] + dummy_cols + cont_cols
need       = ["log_s","price","store","month_idx"] + Xnames
df2        = df.dropna(subset=need).copy()
df2["brand_for_iv"] = brand_key.loc[df2.index].astype("string")

In [10]:
import numpy as np, pandas as pd
from IPython.display import display

def summary_stats_styler(
    df: pd.DataFrame,
    cont_vars=None,
    bin_vars=None,
    var_labels=None,
    percentiles=(0.10, 0.50, 0.90),
    digits=3,
    percent_cols=None
):
    """
    Return a Pandas Styler that renders an econ-journal style summary table.
    Nothing is printed; call display(summary_stats_styler(...)) in Jupyter.
    """
    df = df.copy()

    # Defaults for your dataset
    if cont_vars is None:
        cont_vars = [c for c in ["price","prod_mkt_share","tar_mean","nic_mean","co_mean"] if c in df.columns]
    if bin_vars is None:
        bin_vars  = [c for c in ["dlx","supslim","slim","value","premium","flavored","carton","generic_hardcoded"] if c in df.columns]
    if var_labels is None:
        var_labels = {}
    if percent_cols is None:
        percent_cols = []

    use_vars = cont_vars + bin_vars
    if not use_vars:
        raise ValueError("No variables found. Pass cont_vars/bin_vars explicitly.")

    # numeric coercion
    dnum = df[use_vars].apply(pd.to_numeric, errors="coerce")

    # names for percentiles
    p10, p50, p90 = percentiles
    p10n, p90n = f"p{int(100*p10):02d}", f"p{int(100*p90):02d}"

    rows = []
    for v in use_vars:
        x = dnum[v].dropna()
        if x.empty:
            continue
        rows.append({
            "Variable": var_labels.get(v, v),
            "N": int(x.shape[0]),
            "Mean": x.mean(),
            "Std. Dev.": x.std(ddof=1),
            "Min": x.min(),
            p10n: x.quantile(p10),
            "Median": x.quantile(p50),
            p90n: x.quantile(p90),
            "Max": x.max(),
            "_is_dummy": int(v in bin_vars)  # keep mask separately
        })

    tab = pd.DataFrame(rows, columns=[
        "Variable","N","Mean","Std. Dev.","Min",p10n,"Median",p90n,"Max","_is_dummy"
    ])

    # Build mask BEFORE dropping the helper column
    dummy_mask = tab.set_index("Variable")["_is_dummy"].astype(bool)
    tab = tab.drop(columns=["_is_dummy"]).set_index("Variable")

    # formatters
    percent_set = set(percent_cols)
    def numfmt(val, col=None):
        if pd.isna(val): return ""
        if (col in percent_set): return f"{val*100:.{digits}f}%"
        return f"{val:.{digits}f}"

    sty = (tab
        .style
        .format({"N": "{:,}"})
        .format({c: (lambda v, c=c: numfmt(v, c)) for c in ["Mean","Std. Dev.","Min",p10n,"Median",p90n,"Max"]})
        .set_caption("Summary Statistics")
        .set_table_styles([
            {"selector":"caption", "props":"caption-side: top; font-weight:600; font-size:1.05rem;"},
            {"selector":"th.col_heading","props":"text-align:center;"},
            {"selector":"th.row_heading","props":"text-align:left; white-space:nowrap;"},
            {"selector":"td","props":"text-align:right; padding:4px 10px;"},
        ])
    )

    # subtle shading for dummy rows
    def shade_dummy_rows(row):
        return ["background-color: rgba(0,0,0,0.05)" if dummy_mask.get(row.name, False) else "" for _ in row]
    sty = sty.apply(shade_dummy_rows, axis=1)

    return sty

# --------------------------------------

labels = {
    "price":"Price (per pack)",
    "prod_mkt_share":"Product share",
    "tar_mean":"Tar (mg)",
    "nic_mean":"Nicotine (mg)",
    "co_mean":"CO (mg)",
    "dlx":"Deluxe", "supslim":"Super slim", "slim":"Slim",
    "value":"Value line", "premium":"Premium", "flavored":"Flavored",
    "carton":"Carton", "generic_hardcoded":"Generic",
}
cont_vars = [c for c in ["price","prod_mkt_share","tar_mean","nic_mean","co_mean"] if c in df2.columns]
bin_vars  = [c for c in ["dlx","supslim","slim","value","premium","flavored","carton","generic_hardcoded"] if c in df2.columns]

display(summary_stats_styler(
    df2,
    cont_vars=cont_vars,
    bin_vars=bin_vars,
    var_labels=labels,
    percentiles=(0.25, 0.50, 0.75),
    digits=3,
    percent_cols=["prod_mkt_share"]   # show share as %
))


,N,Mean,Std. Dev.,Min,p25,Median,p75,Max
Variable,,,,,,,,
Price (per pack),731769,2.253,0.709,0.035,1.987,2.240,2.500,250.000
Product share,731769,0.003,0.012,0.000,0.001,0.001,0.003,0.478
Tar (mg),731769,14.221,1.464,9.000,14.527,15.000,15.000,20.000
Nicotine (mg),731769,0.901,0.071,0.727,0.900,0.900,0.900,1.300
CO (mg),731769,13.102,0.313,10.800,13.122,13.122,13.122,15.067
Deluxe,731769,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Super slim,731769,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Slim,731769,0.009,0.096,0.000,0.000,0.000,0.000,1.000
Value line,731769,0.005,0.071,0.000,0.000,0.000,0.000,1.000


In [27]:
import numpy as np, pandas as pd
from IPython.display import display

df = df2.copy()

# --- Choose brand column & normalize ---
brand_col = "brand_clean" if "brand_clean" in df.columns else ("brand" if "brand" in df.columns else None)
brand_raw = df[brand_col].astype("string").str.strip() if brand_col else pd.Series("", index=df.index)

brand_key = brand_raw.str.lower()
brand_key = brand_key.where(~(brand_key.eq("") | brand_key.isna()), "generic")

# generic flags (any one implies "generic")
is_generic = (
    pd.to_numeric(df.get("generic_hardcoded", 0), errors="coerce").fillna(0).astype(int).gt(0)
    | brand_key.eq("generic")
    | df.get("prod_id", pd.Series("", index=df.index)).astype("string").str.lower().eq("generic")
)
brand_key = brand_key.mask(is_generic, "generic")

# display label: keep original case when available; force "Generic" label
brand_disp = brand_raw.where(~brand_key.eq("generic"), "Generic")
df["brand_group"] = brand_key
df["brand_label"] = brand_disp

# --- Ensure quantity / revenue columns exist ---
qty_col = "total_packs" if "total_packs" in df.columns else ("pack_sales" if "pack_sales" in df.columns else None)
rev_col = "total_rev"   if "total_rev"   in df.columns else ("row_revenue" if "row_revenue" in df.columns else None)
if qty_col is None or rev_col is None:
    raise ValueError("Could not find quantity/revenue columns (looked for total_packs/pack_sales and total_rev/row_revenue).")

df[qty_col] = pd.to_numeric(df[qty_col], errors="coerce").fillna(0.0)
df[rev_col] = pd.to_numeric(df[rev_col], errors="coerce").fillna(0.0)

# --- Aggregate by brand (use normalized key; keep a representative label) ---
first_label = (df.groupby("brand_group", observed=True)["brand_label"]
                 .agg(lambda s: s.dropna().iloc[0] if s.notna().any() else ""))
tab = (df.groupby("brand_group", observed=True)
         .agg(n_obs=("brand_group","size"),
              sales_packs=(qty_col,"sum"),
              sales_rev=(rev_col,"sum"))
         .join(first_label.rename("Brand"))
         .reset_index(drop=True))

# Shares & price
tab["obs_share"]   = tab["n_obs"]       / float(df.shape[0])
tab["packs_share"] = tab["sales_packs"] / float(tab["sales_packs"].sum())
tab["rev_share"]   = tab["sales_rev"]   / float(tab["sales_rev"].sum())
tab["avg_price_per_pack"] = np.where(tab["sales_packs"]>0,
                                     tab["sales_rev"]/tab["sales_packs"], np.nan)

# Order nicely (largest pack share first)
tab = tab.sort_values(["packs_share","rev_share","n_obs"], ascending=False)

# Select/display columns
cols = ["Brand","n_obs","obs_share","sales_packs","packs_share","sales_rev","rev_share","avg_price_per_pack"]
tab_disp = tab[cols]

# --- Jupyter-friendly styling ---
sty = (tab_disp.style
       .format({
           "n_obs": "{:,}",
           "obs_share": "{:.2%}",
           "sales_packs": "{:,.0f}",
           "packs_share": "{:.2%}",
           "sales_rev": "${:,.0f}",
           "rev_share": "{:.2%}",
           "avg_price_per_pack": "${:,.2f}",
       })
       .set_caption("Brand-Level Observation & Sales Shares (Generic listed separately)")
       .set_table_styles([
           {"selector":"caption","props":"caption-side: top; font-weight:600; font-size:1.05rem;"},
           {"selector":"th.col_heading","props":"text-align:center;"},
           {"selector":"th.row_heading","props":"text-align:left; white-space:nowrap;"},
           {"selector":"td","props":"text-align:right; padding:4px 10px;"},
       ])
)
display(sty)


,Brand,n_obs,obs_share,sales_packs,packs_share,sales_rev,rev_share,avg_price_per_pack
2,Generic,"683,309",93.38%,"22,395,737",95.46%,"$46,658,264",94.82%,$2.08
4,marlboro,"19,840",2.71%,"686,393",2.93%,"$1,688,802",3.43%,$2.46
0,benson & hedges,"11,289",1.54%,"161,344",0.69%,"$378,306",0.77%,$2.34
5,virginia slims,"6,802",0.93%,"101,177",0.43%,"$241,398",0.49%,$2.39
3,kool,"7,154",0.98%,"64,126",0.27%,"$160,664",0.33%,$2.51
6,winston,"2,552",0.35%,"48,124",0.21%,"$70,656",0.14%,$1.47
7,winston select,822,0.11%,"4,927",0.02%,"$9,930",0.02%,$2.02
1,doral,1,0.00%,1,0.00%,$4,0.00%,$4.28


# $\text{Benchmark: Simple Logit}$

In [25]:
import numpy as np, pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS

# -----------------------
# helpers
# -----------------------

def demean_within(frame, cols, keys):
    means = frame.groupby(keys, observed=True)[cols].transform("mean")
    return frame[cols] - means

def prune_instruments_for_full_rank(exog_df, Z_df, tol=1e-10):
    W, keep = exog_df.copy(), []
    for c in Z_df.columns:
        r_old = np.linalg.matrix_rank(W.to_numpy(), tol)
        W_try = pd.concat([W, Z_df[[c]]], axis=1)
        r_new = np.linalg.matrix_rank(W_try.to_numpy(), tol)
        if r_new > r_old:
            keep.append(c); W = W_try
    return Z_df[keep]

def standardize_cols(df):
    out = df.copy()
    for c in out.columns:
        s = float(out[c].std(skipna=True))
        if np.isfinite(s) and s > 0:
            out[c] = out[c] / s
    return out
    
# ========================
# 1) regressors
# ========================
dummy_cols = [c for c in ["dlx","supslim","slim","value","premium","flavored","carton"] if c in df.columns]
cont_cols  = [c for c in ["tar_mean","nic_mean","co_mean"] if c in df.columns]
Xnames     = ["price"] + cont_cols
need       = ["log_s","price","store","month_idx"] + Xnames
df2        = df.dropna(subset=need).copy()
df2["brand_for_iv"] = brand_key.loc[df2.index].astype("string")

# ========================
# 2) instruments
# ========================
USE_BLP_IVS = True     # set False for Hausman-only

# Hausman (coalesced z1 -> z2 -> z3), raw
g_uq = df2.groupby(["prod_id","month_idx"], observed=True)
cnt_uq = g_uq["price"].transform("count"); sum_uq = g_uq["price"].transform("sum")
z1_raw = np.where(cnt_uq.gt(1), (sum_uq - df2["price"]) / (cnt_uq - 1), np.nan)

g_u  = df2.groupby(["prod_id"], observed=True)
cnt_u = g_u["price"].transform("count"); sum_u = g_u["price"].transform("sum")
g_us = df2.groupby(["prod_id","store"], observed=True)
cnt_us = g_us["price"].transform("count"); sum_us = g_us["price"].transform("sum")
z2_raw = np.where((cnt_u - cnt_us).gt(0), (sum_u - sum_us) / (cnt_u - cnt_us), np.nan)

gbm  = df2.groupby(["brand_for_iv","month_idx"], observed=True)
cnt_bm = gbm["price"].transform("count"); sum_bm = gbm["price"].transform("sum")
gbms = df2.groupby(["brand_for_iv","month_idx","store"], observed=True)
cnt_bms = gbms["price"].transform("count"); sum_bms = gbms["price"].transform("sum")
z3_raw = np.where((cnt_bm - cnt_bms).gt(0), (sum_bm - sum_bms) / (cnt_bm - cnt_bms), np.nan)

z_haus_raw = pd.Series(z1_raw, index=df2.index)
z_haus_raw = z_haus_raw.where(z_haus_raw.notna(), pd.Series(z2_raw, index=df2.index))
z_haus_raw = z_haus_raw.where(z_haus_raw.notna(), pd.Series(z3_raw, index=df2.index))

Z_raw = pd.DataFrame({"z_haus": z_haus_raw}, index=df2.index)

# Optional BLP(cont) instruments, raw
if USE_BLP_IVS and len(cont_cols) > 0:
    gm  = df2.groupby(mkt, observed=True)
    gfb = df2.groupby(mkt + ["brand_for_iv"], observed=True)
    for c in cont_cols:
        tot = gm[c].transform("sum")
        own = gfb[c].transform("sum")
        Z_raw[f"iv_riv_sum_{c}"] = (tot - own)
        Z_raw[f"iv_own_sum_{c}"] = (own - df2[c])
    # optional rival count
    Z_raw["iv_rival_count"] = gm["price"].transform("size") - gfb["price"].transform("size")

# ========================
# 3) Demeaning
# ========================
# y and X
X_tilde = demean_within(df2, Xnames, mkt)
y_tilde = (df2["log_s"] - df2.groupby(mkt, observed=True)["log_s"].transform("mean")).rename("log_s")

# Z (demean the whole instrument matrix at once)
Z_tilde = demean_within(pd.concat([df2[mkt], Z_raw], axis=1), list(Z_raw.columns), mkt)

# align single estimation sample
all_parts = pd.concat([y_tilde, X_tilde, Z_tilde], axis=1).replace([np.inf,-np.inf], np.nan).dropna()
y_iv  = all_parts["log_s"]
X_iv  = all_parts[Xnames]
Z_iv  = all_parts[Z_tilde.columns]

# drop zero-variance columns post-demean
X_iv = X_iv.loc[:, X_iv.apply(lambda s: np.nanstd(s.to_numpy()) > 0)]
Z_iv = Z_iv.loc[:, Z_iv.apply(lambda s: np.nanstd(s.to_numpy()) > 0)]

# standardize instruments, then prune for rank
Z_iv = standardize_cols(Z_iv)
exog = X_iv.drop(columns=["price"])
Z_iv = prune_instruments_for_full_rank(exog, Z_iv)

clusters_iv = pd.to_numeric(df2.loc[all_parts.index, "store"], errors="coerce").astype(int).to_numpy()

# ========================
# 4) Estimates
# ========================
# OLS (with FE absorbed) — optional
ols = sm.OLS(y_iv, X_iv).fit(cov_type="cluster", cov_kwds={"groups": clusters_iv})
print("\n[Simple Logit OLS | Market FE absorbed]")
print(ols.summary().tables[1])

# IV (2SLS)
iv = IV2SLS(
    dependent=y_iv,
    exog=exog,                          # demeaned exogenous regressors
    endog=X_iv[["price"]],              # demeaned endogenous regressor
    instruments=Z_iv                    # demeaned instruments
).fit(cov_type="clustered", clusters=clusters_iv)

print("\n[Simple Logit IV | Market FE | Hausman{}]".format(" + BLP(cont)" if USE_BLP_IVS else ""))
print(iv.summary)
try:
    print("\n[First stage]"); print(iv.first_stage.summary)
except Exception:
    pass



[Simple Logit OLS | Market FE absorbed]
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
price         -0.0310      0.029     -1.052      0.293      -0.089       0.027
tar_mean      -0.3596      0.011    -31.398      0.000      -0.382      -0.337
nic_mean       5.0089      0.168     29.867      0.000       4.680       5.338
co_mean        0.0414      0.013      3.309      0.001       0.017       0.066

[Simple Logit IV | Market FE | Hausman + BLP(cont)]
                          IV-2SLS Estimation Summary                          
Dep. Variable:                  log_s   R-squared:                      0.0589
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0589
No. Observations:              731768   F-statistic:                    1460.5
Date:                Mon, Nov 03 2025   P-value (F-stat)                0.0000
Time:                        09:40:39

In [26]:
# ========================
# 4) Present Brand × Brand elasticities nicely
# ========================
alpha  = float(iv.params.get("price", np.nan))
beta_f = float(iv.params.get("flavored", 0.0))   # 0 if not in Xnames

df_calc = df2.loc[df2.index].copy()
df_calc["brand_key"] = brand_key.loc[df_calc.index].astype("string")
brands = sorted(df_calc["brand_key"].unique().tolist())

E_price_sum={}; E_price_cnt={}; E_flav_sum={}; E_flav_cnt={}
for b in brands:
    for c in brands:
        E_price_sum[(b,c)]=0.0; E_price_cnt[(b,c)]=0
        E_flav_sum [(b,c)]=0.0; E_flav_cnt [(b,c)]=0

for key, g in df_calc.groupby(mkt, observed=True):
    s = g["prod_mkt_share"].to_numpy()
    p = g["price"].to_numpy()
    B = g["brand_key"].to_numpy()
    if s.size < 2: continue

    Jp = alpha * (np.diag(s) - np.outer(s, s))
    if "flavored" in g.columns:
        f = g["flavored"].to_numpy().astype(float)
        Jf = beta_f * (np.diag(s * f) - np.outer(s, s * f))
    else:
        Jf = np.zeros((s.size, s.size))
    JpP = Jp * p[None, :]  # % change

    brands_here = np.unique(B)
    S_b = {bk: float(s[B==bk].sum()) for bk in brands_here}

    for b in brands_here:
        Sb = S_b[b]
        if Sb <= 0: continue
        rows = (B == b)
        rJpP = JpP[rows, :].sum(axis=0)
        rJf  = Jf [rows, :].sum(axis=0)
        for c in brands_here:
            cols = (B == c)
            E_bc_price = (rJpP[cols].sum()) / Sb
            E_bc_flav  = - rJf[cols].sum()
            E_price_sum[(b,c)] += E_bc_price; E_price_cnt[(b,c)] += 1
            E_flav_sum [(b,c)] += E_bc_flav;  E_flav_cnt [(b,c)] += 1

def avg_matrix(sum_d, cnt_d, brands):
    M = np.full((len(brands), len(brands)), np.nan)
    for i,b in enumerate(brands):
        for j,c in enumerate(brands):
            if cnt_d[(b,c)] > 0:
                M[i,j] = sum_d[(b,c)] / cnt_d[(b,c)]
    return pd.DataFrame(M, index=brands, columns=brands)
    
E_price = avg_matrix(E_price_sum, E_price_cnt, brands)  # %
E_flav  = avg_matrix(E_flav_sum,  E_flav_cnt,  brands)  # share levels

# 1) Ensure consistent brand order and build a % version for price
brands = sorted(E_price.index.tolist())
E_price = E_price.reindex(index=brands, columns=brands)
E_flav  = E_flav .reindex(index=brands, columns=brands)
E_price_pct = E_price * 100.0

# 2) Continuous (long/tidy) table — prints once without "row-by-row" updates
elasticities_long = (
    E_price_pct.stack().rename("price_elasticity_pct")
    .to_frame()
    .join(E_flav.stack().rename("flavored_semi_delta"))
    .rename_axis(index=["affected_brand","shocked_brand"])
    .reset_index()
    .sort_values(["affected_brand","shocked_brand"], kind="stable")
)

# Pretty print in console
pd.set_option("display.width", 160)
pd.set_option("display.max_rows", 100000)
print("\nBrand × Brand elasticities (continuous long format)")
print(elasticities_long.to_string(
    index=False,
    formatters={
        "price_elasticity_pct": lambda v: f"{v:8.3f}",   # % ΔS_b for 1% ↑ price of brand c
        "flavored_semi_delta":  lambda v: f"{v: .5f}",   # ΔS_b when brand c flavored is banned
    }
))

# 3) Nicer wide tables for notebooks (optional)
try:
    from IPython.display import display
    # price elasticities
    display(
        E_price_pct.style
            .format("{:.2f}%")
            .background_gradient(cmap="coolwarm", axis=None)
            .set_caption("Brand × Brand PRICE Elasticities (% ΔS_b for 1% ↑ price of brand c)")
            .set_table_styles([
                {"selector": "th.col_heading", "props": "text-align:center;"},
                {"selector": "th.row_heading", "props": "text-align:right;"},
            ])
    )
    # flavored semi-elasticities
    display(
        E_flav.style
            .format("{:.4f}")
            .background_gradient(cmap="Greens", axis=None)
            .set_caption("Brand × Brand FLAVORED Semi-Elasticities (ΔS_b when brand c flavored is banned)")
            .set_table_styles([
                {"selector": "th.col_heading", "props": "text-align:center;"},
                {"selector": "th.row_heading", "props": "text-align:right;"},
            ])
    )
except Exception:
    # safe no-op if running outside a notebook
    pass


Brand × Brand elasticities (continuous long format)
 affected_brand   shocked_brand price_elasticity_pct flavored_semi_delta
benson & hedges benson & hedges              -83.619             0.00000
benson & hedges         generic               14.206             0.00000
benson & hedges            kool                0.521             0.00000
benson & hedges        marlboro                5.517             0.00000
benson & hedges  virginia slims                0.814             0.00000
benson & hedges         winston                0.087             0.00000
benson & hedges  winston select                0.029             0.00000
          doral           doral             -150.670             0.00000
          doral         generic                6.116             0.00000
        generic benson & hedges                1.226             0.00000
        generic           doral                0.025             0.00000
        generic         generic              -59.779             0.0000

,benson & hedges,doral,generic,kool,marlboro,virginia slims,winston,winston select
benson & hedges,-83.62%,nan%,14.21%,0.52%,5.52%,0.81%,0.09%,0.03%
doral,nan%,-150.67%,6.12%,nan%,nan%,nan%,nan%,nan%
generic,1.23%,0.03%,-59.78%,0.52%,5.45%,0.81%,0.41%,0.16%
kool,1.24%,nan%,14.04%,-90.27%,5.51%,0.82%,0.09%,0.03%
marlboro,1.23%,nan%,14.04%,0.52%,-82.63%,0.81%,0.09%,0.03%
virginia slims,1.26%,nan%,14.62%,0.53%,5.69%,-86.09%,0.09%,0.03%
winston,1.68%,nan%,21.56%,0.65%,7.88%,1.12%,-54.16%,0.23%
winston select,1.24%,nan%,22.48%,0.48%,5.03%,0.80%,0.07%,-73.73%


,benson & hedges,doral,generic,kool,marlboro,virginia slims,winston,winston select
benson & hedges,0.0000,nan,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
doral,nan,0.0000,0.0000,nan,nan,nan,nan,nan
generic,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
kool,0.0000,nan,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
marlboro,0.0000,nan,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
virginia slims,0.0000,nan,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
winston,0.0000,nan,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
winston select,0.0000,nan,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [19]:
# Use the final estimation sample
diag = df2.loc[df2.index].copy()
diag["brand_key"] = brand_key.loc[diag.index].astype("string")

# Brand-level flavored coverage
brand_cov = (diag.groupby("brand_key", observed=True)
               .agg(n_obs=("flavored","size"),
                    n_markets=("month_idx", lambda s: diag.loc[s.index, ["store","month_idx"]]
                                              .drop_duplicates().shape[0]),
                    frac_flavored=("flavored","mean"),
                    avg_share=("prod_mkt_share","mean"))
               .sort_values(["frac_flavored","avg_share"], ascending=[False,False]))
print("\n[Brand-level flavored coverage]")
print(brand_cov.to_string(float_format=lambda x: f"{x:,.3f}"))

# Within-market variation: share of markets that have at least one flavored=1 and one flavored=0
mk = diag[mkt].astype(str).agg('|'.join, axis=1)
var_by_mkt = (diag.assign(mk=mk)
                .groupby("mk", observed=True)["flavored"]
                .agg(lambda s: (s.max() > 0) and (s.min() == 0)))
print("\nShare of markets with BOTH flavored and non-flavored present:",
      float(var_by_mkt.mean()))



[Brand-level flavored coverage]
                  n_obs  n_markets  frac_flavored  avg_share
brand_key                                                   
kool               7154       1109          1.000      0.001
doral                 1          1          1.000      0.000
benson & hedges   11289       1116          0.668      0.001
marlboro          19840       1130          0.000      0.004
generic          683309       8194          0.000      0.003
winston            2552        881          0.000      0.003
virginia slims     6802       1078          0.000      0.002
winston select      822        369          0.000      0.001

Share of markets with BOTH flavored and non-flavored present: 0.13766170368562364


In [56]:
# ======================
# Counterfactuals (fixed)
# ======================
alpha  = float(iv.params.get("price", np.nan))
beta_f = float(iv.params.get("flavored", 0.0))

mk_cols = ["store","month_idx"]          # market keys (same as mkt)
sim = df2.loc[df2.index].copy()          # estimation sample
sim["brand_key"] = brand_key.loc[sim.index].astype("string")

# Mean utilities from simple logit inversion (already computed for the fit)
delta_base = sim["log_s"].to_numpy()         # δ_jm = log s_jm − log s_0m

def delta_to_shares(df_local, delta_vec):
    """
    Convert δ to predicted shares: s_jm = exp(δ_jm) / (1 + sum_k exp(δ_km)).
    df_local: DataFrame of the rows being predicted (may be a subset).
    delta_vec: NumPy array aligned to df_local.index.
    """
    # market key per row
    mk_key = df_local[mk_cols].astype(str).agg('|'.join, axis=1)
    # safe exp -> Series aligned with df_local
    e = pd.Series(np.exp(np.clip(delta_vec, -700, 700)), index=df_local.index)
    den = e.groupby(mk_key, observed=True).transform("sum") + 1.0
    return e / den  # Series aligned to df_local.index

# --- Policy A: attribute removal (flavored -> 0) ---
d_delta_A = - beta_f * sim.get("flavored", 0).to_numpy()
sim["s_A"] = delta_to_shares(sim, delta_base + d_delta_A)

# --- Policy B: ban flavored SKUs (remove products) ---
keep_B = (sim.get("flavored", 0).to_numpy() == 0)
s_B = pd.Series(0.0, index=sim.index)
s_B.loc[keep_B] = delta_to_shares(sim.loc[keep_B], delta_base[keep_B])
sim["s_B"] = s_B

# ---------- Aggregate to brand-level (market-averaged) ----------
def brand_market_avg(df_local, col):
    g = (df_local.groupby(mk_cols + ["brand_key"], observed=True)[col]
                  .sum()
                  .reset_index())
    return (g.groupby("brand_key", observed=True)[col]
             .mean()
             .sort_values(ascending=False))

brand_base = brand_market_avg(sim, "prod_mkt_share")
brand_A    = brand_market_avg(sim, "s_A")
brand_B    = brand_market_avg(sim, "s_B")

res_A = pd.concat([brand_base.rename("baseline"),
                   brand_A.rename("policy_A"),
                   (brand_A - brand_base).rename("Δshare_attrRemoval")], axis=1)

res_B = pd.concat([brand_base.rename("baseline"),
                   brand_B.rename("policy_B"),
                   (brand_B - brand_base).rename("Δshare_ban")], axis=1)

print("\n[Brand shares | Policy A: flavored attribute removed (flavored→0)]")
print(res_A.to_string(float_format=lambda x: f"{x: .4f}"))

print("\n[Brand shares | Policy B: flavored SKUs banned]")
print(res_B.to_string(float_format=lambda x: f"{x: .4f}"))

# ---------- Outside share comparison ----------
def outside_share(df_local, col):
    s0m = 1.0 - df_local.groupby(mk_cols, observed=True)[col].sum()
    return float(s0m.mean())

s0_base = outside_share(sim, "prod_mkt_share")
s0_A    = outside_share(sim, "s_A")
s0_B    = outside_share(sim, "s_B")
print(f"\nOutside share (market-avg): baseline={s0_base:.4f} | Policy A={s0_A:.4f} | Policy B={s0_B:.4f}")



[Brand shares | Policy A: flavored attribute removed (flavored→0)]
                 baseline  policy_A  Δshare_attrRemoval
brand_key                                              
marlboro           0.0623    0.0622             -0.0001
benson & hedges    0.0147    0.0153              0.0005
virginia slims     0.0096    0.0096             -0.0000
winston            0.0084    0.0084             -0.0000
kool               0.0058    0.0062              0.0004
winston select     0.0022    0.0022             -0.0000
doral              0.0002    0.0002              0.0000

[Brand shares | Policy B: flavored SKUs banned]
                 baseline  policy_B  Δshare_ban
brand_key                                      
marlboro           0.0623    0.0635      0.0012
benson & hedges    0.0147    0.0078     -0.0069
virginia slims     0.0096    0.0098      0.0002
winston            0.0084    0.0084      0.0000
kool               0.0058    0.0000     -0.0058
winston select     0.0022    0.0022      0.

# $\text{Alternatives: Other Specifications}$

## Nested Logit

In [35]:
import numpy as np, pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS

# -----------------------
# helpers
# -----------------------
def series_flag(df, col, dtype="int8"):
    if col in df.columns:
        return pd.to_numeric(df[col], errors="coerce").fillna(0).astype(dtype)
    return pd.Series(0, index=df.index, dtype=dtype)

def demean_within(frame, cols, keys):
    means = frame.groupby(keys, observed=True)[cols].transform("mean")
    return frame[cols] - means

def prune_instruments_for_full_rank(exog_df, Z_df, tol=1e-10):
    W, keep = exog_df.copy(), []
    for c in Z_df.columns:
        r_old = np.linalg.matrix_rank(W.to_numpy(), tol)
        W_try = pd.concat([W, Z_df[[c]]], axis=1)
        r_new = np.linalg.matrix_rank(W_try.to_numpy(), tol)
        if r_new > r_old:
            keep.append(c); W = W_try
    return Z_df[keep]

def standardize_cols(df):
    out = df.copy()
    for c in out.columns:
        s = float(out[c].std(skipna=True))
        if np.isfinite(s) and s > 0:
            out[c] = out[c] / s
    return out

# ========================
# 0) data & keys
# ========================
df = prod_market_m.copy()
df = df.rename(columns={"avg_pack_price": "price"})
mkt = ["store","month_idx"]

# inside/outside shares
sum_inside = df.groupby(mkt, observed=True)["prod_mkt_share"].transform("sum")
df = df[(df["prod_mkt_share"] > 0) & (sum_inside < 1)].copy()
df["s0"] = np.clip(1.0 - sum_inside, 1e-12, 1 - 1e-12)

# robust product id
if "prod_id" not in df.columns:
    if "upc_norm" in df.columns:
        df["prod_id"] = df["upc_norm"].astype("string")
    elif "upc" in df.columns:
        df["prod_id"] = df["upc"].astype("string").str.replace(r"\D","", regex=True)
    else:
        df["prod_id"] = df.index.astype("string")

# ---------- nest: branded vs generic ----------
brand_raw = df.get("brand", pd.Series("", index=df.index))
brand_key = (brand_raw.astype("string").str.strip().str.lower()
             .mask(lambda s: s.eq("") | s.isna(), "generic"))
is_generic = (series_flag(df, "generic_hardcoded") > 0) | brand_key.eq("generic")
brand_key  = brand_key.mask(is_generic, "generic")
df["nest"] = np.where(brand_key.eq("generic"), "generic", "branded")

# ---------- firm_id for IVs ----------
# For generics, use UPC manufacturer prefix (first 5 digits).
# For branded, use the brand (lower-cased).
upc_str = df.get("upc_norm", df["prod_id"]).astype("string")
manuf5  = upc_str.str.slice(0, 5).fillna("")
firm_id = np.where(brand_key.eq("generic"), manuf5, brand_key)
firm_id = pd.Series(firm_id, index=df.index).replace("", "unk").astype("string")
df["firm_id"] = firm_id

# ========================
# 1) nested-logit shares (RAW)
# ========================
df["ln_s"] = np.log(df["prod_mkt_share"]) - np.log(df["s0"])
sg = df.groupby(mkt + ["nest"], observed=True)["prod_mkt_share"].transform("sum")
df = df[sg > 0].copy()
df["ln_s_within"] = np.log(df["prod_mkt_share"]) - np.log(sg)

# regressors
dummy_cols = [c for c in ["dlx","supslim","slim","value","premium","flavored","carton"] if c in df.columns]
cont_cols  = [c for c in ["tar_mean","nic_mean","co_mean"] if c in df.columns]
Xnames     = ["price","ln_s_within"] + dummy_cols + cont_cols

need = ["ln_s","price","store","month_idx","nest","prod_id","firm_id"] + Xnames
df2  = df.dropna(subset=need).copy()

# ========================
# 2) instruments (RAW) – Hausman + BLP(cont w/ firm) + Nest(cont)
# ========================
Z_raw = pd.DataFrame(index=df2.index)

# --- Hausman (coalesced) ---
# z1: product×month, other-store mean price
g_uq = df2.groupby(["prod_id","month_idx"], observed=True)
cnt_uq = g_uq["price"].transform("count"); sum_uq = g_uq["price"].transform("sum")
z1 = np.where(cnt_uq.gt(1), (sum_uq - df2["price"]) / (cnt_uq - 1), np.nan)

# zf: firm×month, other-store mean price
g_fm  = df2.groupby(["firm_id","month_idx"], observed=True)
cnt_fm = g_fm["price"].transform("count"); sum_fm = g_fm["price"].transform("sum")
g_fms = df2.groupby(["firm_id","month_idx","store"], observed=True)
cnt_fms = g_fms["price"].transform("count"); sum_fms = g_fms["price"].transform("sum")
zf = np.where((cnt_fm - cnt_fms).gt(0), (sum_fm - sum_fms) / (cnt_fm - cnt_fms), np.nan)

# z2: product across all months, other-store mean price
g_u  = df2.groupby(["prod_id"], observed=True)
cnt_u = g_u["price"].transform("count"); sum_u = g_u["price"].transform("sum")
g_us = df2.groupby(["prod_id","store"], observed=True)
cnt_us = g_us["price"].transform("count"); sum_us = g_us["price"].transform("sum")
z2 = np.where((cnt_u - cnt_us).gt(0), (sum_u - sum_us) / (cnt_u - cnt_us), np.nan)

# zn: nest×month, other-store mean price
g_nm  = df2.groupby(["nest","month_idx"], observed=True)
cnt_nm = g_nm["price"].transform("count"); sum_nm = g_nm["price"].transform("sum")
g_nms = df2.groupby(["nest","month_idx","store"], observed=True)
cnt_nms = g_nms["price"].transform("count"); sum_nms = g_nms["price"].transform("sum")
zn = np.where((cnt_nm - cnt_nms).gt(0), (sum_nm - sum_nms) / (cnt_nm - cnt_nms), np.nan)

z_haus_raw = pd.Series(z1, index=df2.index)
for alt in (zf, z2, zn):
    z_haus_raw = z_haus_raw.where(z_haus_raw.notna(), pd.Series(alt, index=df2.index))
Z_raw["z_haus"] = z_haus_raw

# --- BLP(cont) by FIRM (robust when brands sparse) ---
gm  = df2.groupby(mkt, observed=True)
gff = df2.groupby(mkt + ["firm_id"], observed=True)
for c in cont_cols:
    tot = gm[c].transform("sum")
    own = gff[c].transform("sum")
    Z_raw[f"iv_firm_riv_sum_{c}"] = tot - own
    Z_raw[f"iv_firm_own_sum_{c}"] = own - df2[c]

# counts (help first stage)
Z_raw["iv_rival_count"]      = gm["price"].transform("size") - 1
Z_raw["iv_firm_rival_count"] = gm["price"].transform("size") - gff["price"].transform("size")

# --- Nest(cont) proxies for ln_s_within (same-nest & other-nest) ---
gmn = df2.groupby(mkt + ["nest"], observed=True)
for c in cont_cols:
    nest_tot = gmn[c].transform("sum")
    Z_raw[f"iv_nest_same_sum_{c}"]  = nest_tot - df2[c]
    Z_raw[f"iv_nest_other_sum_{c}"] = gm[c].transform("sum") - nest_tot
Z_raw["iv_nest_same_cnt"]  = gmn["price"].transform("size") - 1
Z_raw["iv_nest_other_cnt"] = gm["price"].transform("size") - gmn["price"].transform("size")

# ========================
# 3) ONE FE removal for y, X, Z
# ========================
X_tilde = demean_within(df2, Xnames, mkt)
y_tilde = (df2["ln_s"] - df2.groupby(mkt, observed=True)["ln_s"].transform("mean")).rename("ln_s")
Z_tilde = demean_within(pd.concat([df2[mkt], Z_raw], axis=1), list(Z_raw.columns), mkt)

# single estimation sample
all_parts = pd.concat([y_tilde, X_tilde, Z_tilde], axis=1).replace([np.inf,-np.inf], np.nan).dropna()
y_iv  = all_parts["ln_s"]
X_iv  = all_parts[Xnames]
Z_iv  = all_parts[Z_tilde.columns]

# drop zero-variance; standardize; prune for rank
X_iv = X_iv.loc[:, X_iv.apply(lambda s: np.nanstd(s.to_numpy()) > 0)]
Z_iv = standardize_cols(Z_iv.loc[:, Z_iv.apply(lambda s: np.nanstd(s.to_numpy()) > 0)])

# endogenous: price & ln_s_within
exog = X_iv.drop(columns=["price","ln_s_within"])
Z_iv = prune_instruments_for_full_rank(exog, Z_iv)

clusters_iv = pd.to_numeric(df2.loc[all_parts.index, "store"], errors="coerce").astype(int).to_numpy()

# ========================
# 4) Estimation
# ========================
ols = sm.OLS(y_iv, X_iv).fit(cov_type="cluster", cov_kwds={"groups": clusters_iv})
print("\n[OLS Nested Logit | Market FE absorbed]")
print(ols.summary().tables[1])

iv = IV2SLS(
    dependent=y_iv,
    exog=exog,
    endog=X_iv[["price","ln_s_within"]],
    instruments=Z_iv
).fit(cov_type="clustered", clusters=clusters_iv)
print("\n[IV Nested Logit | Market FE absorbed | Hausman + BLP(cont, firm) + Nest(cont)]")
print(iv.summary)
try:
    print("\n[First stages]"); print(iv.first_stage.summary)
except Exception:
    pass

# quick elasticity diagnostic
sigma = float(iv.params.get("ln_s_within", np.nan))
alpha = float(iv.params.get("price", np.nan))
sbar   = float(df2.loc[all_parts.index, "prod_mkt_share"].mean())
sjgbar = float((df2.loc[all_parts.index, "prod_mkt_share"] /
                df2.loc[all_parts.index].groupby(mkt + ["nest"], observed=True)["prod_mkt_share"].transform("sum")).mean())
pbar   = float(df2.loc[all_parts.index, "price"].mean())
eps    = -alpha * pbar * (1 - sigma*(1 - sjgbar) - sbar)
print(f"\nσ (nesting): {sigma:.3f} | implied avg own-price elasticity ≈ {eps:.2f}")



[OLS Nested Logit | Market FE absorbed]
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
price        1.028e-15   9.78e-16      1.052      0.293   -8.88e-16    2.94e-15
ln_s_within     1.0000   2.03e-15   4.93e+14      0.000       1.000       1.000
value        6.939e-17   6.63e-15      0.010      0.992   -1.29e-14    1.31e-14
carton      -1.125e-15   3.64e-16     -3.085      0.002   -1.84e-15    -4.1e-16
tar_mean     1.986e-16    4.7e-16      0.422      0.673   -7.23e-16    1.12e-15
nic_mean     1.108e-14   8.63e-15      1.284      0.199   -5.83e-15     2.8e-14
co_mean      1.688e-15   3.56e-15      0.474      0.635   -5.29e-15    8.66e-15

[IV Nested Logit | Market FE absorbed | Hausman + BLP(cont, firm) + Nest(cont)]
                          IV-2SLS Estimation Summary                          
Dep. Variable:                   ln_s   R-squared:                      1.0000


In [43]:
# ========================
# 4) Present Brand × Brand elasticities nicely
# ========================
alpha  = float(iv.params.get("price", np.nan))
beta_f = float(iv.params.get("flavored", 0.0))   # 0 if not in Xnames

df_calc = df2.loc[df2.index].copy()
df_calc["brand_key"] = brand_key.loc[df_calc.index].astype("string")
brands = sorted(df_calc["brand_key"].unique().tolist())

E_price_sum={}; E_price_cnt={}; E_flav_sum={}; E_flav_cnt={}
for b in brands:
    for c in brands:
        E_price_sum[(b,c)]=0.0; E_price_cnt[(b,c)]=0
        E_flav_sum [(b,c)]=0.0; E_flav_cnt [(b,c)]=0

for key, g in df_calc.groupby(mkt, observed=True):
    s = g["prod_mkt_share"].to_numpy()
    p = g["price"].to_numpy()
    B = g["brand_key"].to_numpy()
    if s.size < 2: continue

    Jp = alpha * (np.diag(s) - np.outer(s, s))
    if "flavored" in g.columns:
        f = g["flavored"].to_numpy().astype(float)
        Jf = beta_f * (np.diag(s * f) - np.outer(s, s * f))
    else:
        Jf = np.zeros((s.size, s.size))
    JpP = Jp * p[None, :]  # % change

    brands_here = np.unique(B)
    S_b = {bk: float(s[B==bk].sum()) for bk in brands_here}

    for b in brands_here:
        Sb = S_b[b]
        if Sb <= 0: continue
        rows = (B == b)
        rJpP = JpP[rows, :].sum(axis=0)
        rJf  = Jf [rows, :].sum(axis=0)
        for c in brands_here:
            cols = (B == c)
            E_bc_price = (rJpP[cols].sum()) / Sb
            E_bc_flav  = - rJf[cols].sum()
            E_price_sum[(b,c)] += E_bc_price; E_price_cnt[(b,c)] += 1
            E_flav_sum [(b,c)] += E_bc_flav;  E_flav_cnt [(b,c)] += 1

def avg_matrix(sum_d, cnt_d, brands):
    M = np.full((len(brands), len(brands)), np.nan)
    for i,b in enumerate(brands):
        for j,c in enumerate(brands):
            if cnt_d[(b,c)] > 0:
                M[i,j] = sum_d[(b,c)] / cnt_d[(b,c)]
    return pd.DataFrame(M, index=brands, columns=brands)
    
E_price = avg_matrix(E_price_sum, E_price_cnt, brands)  # %
E_flav  = avg_matrix(E_flav_sum,  E_flav_cnt,  brands)  # share levels

# 1) Ensure consistent brand order and build a % version for price
brands = sorted(E_price.index.tolist())
E_price = E_price.reindex(index=brands, columns=brands)
E_flav  = E_flav .reindex(index=brands, columns=brands)
E_price_pct = E_price * 100.0

# 2) Continuous (long/tidy) table — prints once without "row-by-row" updates
elasticities_long = (
    E_price_pct.stack().rename("price_elasticity_pct")
    .to_frame()
    .join(E_flav.stack().rename("flavored_semi_delta"))
    .rename_axis(index=["affected_brand","shocked_brand"])
    .reset_index()
    .sort_values(["affected_brand","shocked_brand"], kind="stable")
)

# Pretty print in console
pd.set_option("display.width", 160)
pd.set_option("display.max_rows", 100000)
print("\nBrand × Brand elasticities (continuous long format)")
print(elasticities_long.to_string(
    index=False,
    formatters={
        "price_elasticity_pct": lambda v: f"{v:8.3f}",   # % ΔS_b for 1% ↑ price of brand c
        "flavored_semi_delta":  lambda v: f"{v: .5f}",   # ΔS_b when brand c flavored is banned
    }
))

# 3) Nicer wide tables for notebooks (optional)
try:
    from IPython.display import display
    # price elasticities
    display(
        E_price_pct.style
            .format("{:.2f}%")
            .background_gradient(cmap="coolwarm", axis=None)
            .set_caption("Brand × Brand PRICE Elasticities (% ΔS_b for 1% ↑ price of brand c)")
            .set_table_styles([
                {"selector": "th.col_heading", "props": "text-align:center;"},
                {"selector": "th.row_heading", "props": "text-align:right;"},
            ])
    )
    # flavored semi-elasticities
    display(
        E_flav.style
            .format("{:.4f}")
            .background_gradient(cmap="Greens", axis=None)
            .set_caption("Brand × Brand FLAVORED Semi-Elasticities (ΔS_b when brand c flavored is banned)")
            .set_table_styles([
                {"selector": "th.col_heading", "props": "text-align:center;"},
                {"selector": "th.row_heading", "props": "text-align:right;"},
            ])
    )
except Exception:
    # safe no-op if running outside a notebook
    pass


Brand × Brand elasticities (continuous long format)
 affected_brand   shocked_brand price_elasticity_pct flavored_semi_delta
benson & hedges benson & hedges             -236.708            -0.00371
benson & hedges         generic               40.209             0.00000
benson & hedges            kool                1.474             0.00006
benson & hedges        marlboro               15.614             0.00000
benson & hedges  virginia slims                2.305             0.00000
benson & hedges         winston                0.246             0.00000
benson & hedges  winston select                0.081             0.00000
        generic benson & hedges                3.469             0.00082
        generic         generic             -184.504             0.00000
        generic            kool                1.461             0.00065
        generic        marlboro               15.430             0.00000
        generic  virginia slims                2.305             0.0000

,benson & hedges,generic,kool,marlboro,virginia slims,winston,winston select
benson & hedges,-236.71%,40.21%,1.47%,15.61%,2.30%,0.25%,0.08%
generic,3.47%,-184.50%,1.46%,15.43%,2.30%,1.16%,0.44%
kool,3.50%,39.73%,-255.54%,15.58%,2.32%,0.25%,0.08%
marlboro,3.47%,39.74%,1.46%,-233.92%,2.30%,0.25%,0.08%
virginia slims,3.57%,41.39%,1.51%,16.11%,-243.71%,0.25%,0.08%
winston,4.75%,61.03%,1.85%,22.32%,3.16%,-153.31%,0.64%
winston select,3.52%,63.64%,1.35%,14.25%,2.26%,0.19%,-208.71%


,benson & hedges,generic,kool,marlboro,virginia slims,winston,winston select
benson & hedges,-0.0037,0.0000,0.0001,0.0000,0.0000,0.0000,0.0000
generic,0.0008,0.0000,0.0006,0.0000,0.0000,0.0000,0.0000
kool,0.0000,0.0000,-0.0031,0.0000,0.0000,0.0000,0.0000
marlboro,0.0003,0.0000,0.0003,0.0000,0.0000,0.0000,0.0000
virginia slims,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
winston,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
winston select,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


## Mixed Logit

In [207]:
import numpy as np, pandas as pd
import pyblp as blp

# ---------- helpers (pandas-safe) ----------
def series_flag(df, col, dtype="int8"):
    """Return 0/1 Series aligned to df.index; 0 if column missing."""
    if col in df.columns:
        return pd.to_numeric(df[col], errors="coerce").fillna(0).astype(dtype)
    return pd.Series(0, index=df.index, dtype=dtype)

def pick_first(df, names):
    """Return the first existing column name from 'names', else None."""
    for n in names:
        if n in df.columns:
            return n
    return None

# ===========================
# 0) Prep products dataframe
# ===========================
products = df.copy()

# keep cigarettes only (guard)
if "prod_type" in products.columns:
    products = products[products["prod_type"].str.lower().eq("cigarette")].copy()

# ---- robust column mapping ----
# prices & shares
if "prices" not in products.columns:
    if "avg_pack_price" in products.columns: products = products.rename(columns={"avg_pack_price":"prices"})
    elif "price" in products.columns:        products = products.rename(columns={"price":"prices"})

if "shares" not in products.columns and "prod_mkt_share" in products.columns:
    products = products.rename(columns={"prod_mkt_share":"shares"})

# product ids
if "product_ids" not in products.columns:
    if "prod_id" in products.columns:
        products = products.rename(columns={"prod_id":"product_ids"})
    elif "upc_norm" in products.columns:
        products = products.rename(columns={"upc_norm":"product_ids"})
    elif "upc" in products.columns:
        products = products.rename(columns={"upc":"product_ids"})

# ---- build market_ids from store + time ----
time_col = pick_first(products, ["month_idx","month"])
if "store" not in products.columns or time_col is None:
    raise KeyError("Need 'store' and a time index (e.g., 'month_idx' or 'week') to form market_ids.")

products["market_ids"] = (
    products["store"].astype("string").str.strip() + "_" +
    products[time_col].astype("string").str.strip()
)

# clean shares / markets (outside share must be > 0)
products = products[(pd.to_numeric(products["shares"], errors="coerce") > 0)
                    & (pd.to_numeric(products["shares"], errors="coerce") < 0.999)].copy()
outside = 1 - products.groupby("market_ids")["shares"].transform("sum")
products = products[outside > 1e-6].copy()

# exogenous characteristics (present ones only)
X1_cols = [c for c in ["prices","value","premium","tar_mean","nic_mean","co_mean","flavored"]
           if c in products.columns]
for c in X1_cols:
    products[c] = pd.to_numeric(products[c], errors="coerce").fillna(0)

# --------------------------------
# firm_ids: brand for branded; manufacturer prefix for generics
# --------------------------------
if "brand_clean" in products.columns:
    brand = products["brand_clean"].astype("string").str.strip().str.lower().fillna("")
elif "brand" in products.columns:
    brand = products["brand"].astype("string").str.strip().str.lower().fillna("")
else:
    brand = pd.Series("", index=products.index, dtype="string")

pid_str = products["product_ids"].astype("string")
upc_digits = pid_str.str.replace(r"\D", "", regex=True)
manuf5 = upc_digits.str.slice(0, 5).fillna("unk").replace("", "unk")

is_generic = (series_flag(products, "generic") |
              series_flag(products, "generic_hardcoded") |
              series_flag(products, "generic_automated")).astype(bool) | brand.eq("")

# ensure boolean Series aligned to products.index
if not isinstance(is_generic, pd.Series):
    is_generic = pd.Series(is_generic, index=products.index).astype(bool)

# brand for branded, manuf5 for generics
firm_ids = brand.where(~is_generic, manuf5)

# normalize empties / NaNs
firm_ids = firm_ids.astype("string").fillna("unk")
firm_ids = firm_ids.mask(firm_ids.str.strip().eq(""), "unk")

products["firm_ids"] = firm_ids

# =================
# 2) Formulations
# =================
# mean utility (no prices here; price goes into X2 as RC)
X1_form = blp.Formulation('1 ' + ' '.join(f'+ {c}' for c in X1_cols))

# start simple: price as the only random coefficient
X2_form = blp.Formulation('0 + prices')     # later: 'prices + flavored'

pr_integration = blp.Integration("product", size = 5,specification_options={"seed":114514})
integration = blp.Integration("monte_carlo", size=500, specification_options={"seed": 3})
bfgs = blp.Optimization('bfgs', {'gtol': 1e-4})

# ================
# 3) Solve BLP
# ================
problem = blp.Problem((X1_form, X2_form, None), products, integration=pr_integration)

sigma0 = 0.5

results = problem.solve(
    sigma=sigma0,
    optimization=blp.Optimization('l-bfgs-b', {'gtol': 1e-6})
)

print(results)
print("\nbeta (X1 means):\n", results.beta)
print("\nsigma (RC std devs):\n", results.sigma)


Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T      N     F     I     K1    K2    MD 
----  -----  ---  -----  ----  ----  ----
8194  57765  10   40970   8     1     7  

Formulations:
       Column Indices:           0       1       2       3        4         5         6        7    
-----------------------------  ------  ------  -----  -------  --------  --------  -------  --------
 X1: Linear Characteristics      1     prices  value  premium  tar_mean  nic_mean  co_mean  flavored
X2: Nonlinear Characteristics  prices                                                               
Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +5.000000E-01

Nonlinear Coefficient Lower Bounds:
Sigma:     prices    
------  -------------
prices  +0.000000E+00

Nonlinear Coefficient Upper Bounds:
Sigma:     prices    
------  -------------
prices      +INF     

Starting optimization ...



The model may be under-identified. The total number of unfixed parameters is 9, which is more than the total number of moments, 7. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                 
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm      Theta    
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1     00:00:07         0             1          30256        93449        0     +1.117315E-17                 +1.651631E-07  +5.000000E-01

Optimization completed after 00:00:07.
Computing the Hessian and updating the weighting matrix ...
Computed results after 00:00:21.

Problem Results Summary:
GMM     Objective      Projected       Reduced     Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm     Hessian     Shares   Condition Number  Condition Number 
----  -------------  -------------  -------------  -------  ----------------  -----------

In [209]:
X1_form = blp.Formulation('1 + ' + ' + '.join(c for c in X1_cols))
X2_form = blp.Formulation('0 + prices + flavored')
problem  = blp.Problem((X1_form, X2_form, None), products, integration=pr_integration)
sigma0   = np.diag([1.0, 0.5])
results2 = problem.solve(sigma=sigma0, optimization=blp.Optimization('l-bfgs-b', {'gtol':1e-6}))
print(results2)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T      N     F     I      K1    K2    MD 
----  -----  ---  ------  ----  ----  ----
8194  57765  10   204850   8     2     7  

Formulations:
       Column Indices:           0        1        2       3        4         5         6        7    
-----------------------------  ------  --------  -----  -------  --------  --------  -------  --------
 X1: Linear Characteristics      1      prices   value  premium  tar_mean  nic_mean  co_mean  flavored
X2: Nonlinear Characteristics  prices  flavored                                                       
Solving the problem ...

Nonlinear Coefficient Initial Values:
 Sigma:      prices        flavored   
--------  -------------  -------------
 prices   +1.000000E+00               
flavored  +0.000000E+00  +5.000000E-01

Nonlinear Coefficient Lower Bounds:
 Sigma:      prices        flavored   
--------  -------------  -------------
 prices   +0.000000E+00     

The model may be under-identified. The total number of unfixed parameters is 10, which is more than the total number of moments, 7. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm             Theta            
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  ----------------------------
 1     00:00:20         0             1          35048       107429        0     +1.336360E-17                 +3.013818E-07  +1.000000E+00, +5.000000E-01

Optimization completed after 00:00:20.
Computing the Hessian and updating the weighting matrix ...
Computed results after 00:02:13.

Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition

In [211]:
oi = results2.compute_optimal_instruments(method="approximate", seed=7)
prob_optiv = oi.to_problem()
res_optiv = prob_optiv.solve(sigma=sigma0, optimization=blp.Optimization('l-bfgs-b', {'gtol':1e-6}))

Computing optimal instruments for theta ...
Computed optimal instruments after 00:00:04.

Optimal Instrument Results Summary:
Computation  Error Term
   Time        Draws   
-----------  ----------
 00:00:04        1     
Re-creating the problem ...
Re-created the problem after 00:00:00.

Dimensions:
 T      N     F     I      K1    K2    MD 
----  -----  ---  ------  ----  ----  ----
8194  57765  10   204850   8     2     10 

Formulations:
       Column Indices:           0        1        2       3        4         5         6        7    
-----------------------------  ------  --------  -----  -------  --------  --------  -------  --------
 X1: Linear Characteristics      1      prices   value  premium  tar_mean  nic_mean  co_mean  flavored
X2: Nonlinear Characteristics  prices  flavored                                                       
Solving the problem ...

Nonlinear Coefficient Initial Values:
 Sigma:      prices        flavored   
--------  -------------  -------------
 

Detected collinearity issues with [flavored] and at least one other column in ZD. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.
Detected that the 2SLS weighting matrix is nearly singular with condition number +2.067985E+19. To disable singularity checks, set options.singular_tol = numpy.inf.


Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm             Theta            
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  ----------------------------
 1     00:00:09         0             1          35048       107429        0     +2.615088E+13                 +2.916419E+08  +1.000000E+00, +5.000000E-01
 1     00:00:08         0             2          23717        72436        0     +2.154868E+13  +4.602204E+12  +1.409990E+09  +1.055728E-01, +5.278640E-02
 1     00:00:08         0             3          34008       104206        0     +7.852732E+12  +1.369595E+13  +3.265513E+09  +8.477817E-01, +4.238908E-01
 1     00:00:08         0             4    

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +2.095602E+18. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +6.343690E+17. To disable singularity checks, set options.singular_tol = numpy.inf.



Failed to compute standard errors because of invalid estimated covariances of GMM parameters.

Computed results after 00:00:43.

Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  -------------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 1    -3.466086E+13  +3.282192E+09   -3.731071E+17    +9.002831E+16      0      +4.722098E+18      +4.631032E+05  

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm             Theta            
----  -----------  ------------  -----------  -

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +3.006844E+17. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +3.541481E+17. To disable singularity checks, set options.singular_tol = numpy.inf.
